In [1]:
import sys
import asyncio
import time

import numpy as np

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

In [2]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [3]:
#get classes and start them
domain = salobj.Domain()
attcs = ATTCS(domain)
latiss = LATISS(domain)
await asyncio.gather(attcs.start_task, latiss.start_task)

Read historical data in 0.01 sec
Read historical data in 0.21 sec
Read historical data in 0.46 sec
Read historical data in 0.55 sec
Read historical data in 0.94 sec
Read historical data in 1.11 sec
Read historical data in 1.35 sec
Read historical data in 1.45 sec
Read historical data in 1.55 sec
Read historical data in 3.35 sec
Read historical data in 4.39 sec
RemoteTelemetry(ATDome, 0, position) falling behind; read 18 messages


[[None, None, None, None, None, None, None], [None, None, None, None]]

In [4]:
# enable components
await attcs.enable()
await latiss.enable()

Gathering settings.
No settings for atmcs.
Couldn't get settingVersions event. Using empty settings.
No settings for atptg.
Couldn't get settingVersions event. Using empty settings.
No settings for ataos.
Using current from settingVersions event.
No settings for atpneumatics.
Couldn't get settingVersions event. Using empty settings.
No settings for athexapod.
Using current from settingVersions event.
No settings for atdome.
Using test from settingVersions event.
No settings for atdometrajectory.
Using  from settingVersions event.
Settings versions: {'atmcs': '', 'atptg': '', 'ataos': 'current', 'atpneumatics': '', 'athexapod': 'current', 'atdome': 'test', 'atdometrajectory': ''}
Enabling all components
Enabling  atmcs
Enabling  atptg
Enabling  ataos
Enabling  atpneumatics
Enabling  athexapod
Enabling  atdome
Enabling  atdometrajectory
[atmcs]::[<State.ENABLED: 2>]
[atptg]::[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[ataos]::[<State.ENABLED: 2>]
[atp

In [6]:
# send telescope to alt/az/
#await salobj.set_summary_state(attcs.atmcs, salobj.State.ENABLED)
#await attcs.enable()

In [5]:
# point telescope to desired starting position
start_az=10
start_el=70
start_rot_pa=5
await attcs.point_azel(start_az, start_el, rot_pa=start_rot_pa, wait_dome=False)

Sending command
Stop tracking.
Mount tracking state is 1
Tracking state: 1.
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = +054.585 | delta Az= +103.307
[Telescope] delta Alt = +051.087 | delta Az= +099.809
[Telescope] delta Alt = +040.541 | delta Az= +089.252
[Telescope] delta Alt = +035.262 | delta Az= +083.983
[Telescope] delta Alt = +030.018 | delta Az= +078.735
[Telescope] delta Alt = +024.730 | delta Az= +073.448
[Telescope] delta Alt = +019.477 | delta Az= +068.177
[Telescope] delta Alt = +014.230 | delta Az= +062.909
[Telescope] delta Alt = +009.033 | delta Az= +057.679
[Telescope] delta Alt = +003.823 | delta Az= +052.436
[Telescope] delta Alt = +000.321 | delta Az= +047.082
[Telescope] delta Alt = +000.000 | delta Az= +041.860
[Telescope

In [7]:
#declare offset size
d_az= 3.5 # degrees
d_el = 3.5 # degrees
d_rot= 3.5 # degrees

In [8]:
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [9]:
# get RA/DEC of current position
az = Angle(start_az, unit=u.deg)
el = Angle(start_el, unit=u.deg)
time_data = await attcs.atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
#print(curr_time_atptg)
coord_frame_AltAz = AltAz(location=location, obstime=curr_time_atptg)
coord_frame_radec = ICRS()
coord_azel = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec = coord_azel.transform_to(coord_frame_radec)
print('Current Position is: \n {}'.format(coord_azel))
print('Current Position is: \n {}'.format(ra_dec))

# get RA/DEC of target position
az = Angle(start_az+d_az, unit=u.deg)
el = Angle(start_el+d_el, unit=u.deg)
coord_azel_target = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec_target = coord_azel.transform_to(coord_frame_radec)
print('Target Position is: \n {}'.format(coord_azel_target))
print('Target Position is: \n {}'.format(ra_dec_target))


Current Position is: 
 <AltAz Coordinate (obstime=58912.721776106686, location=(1819093.56876225, -5208411.6827961, -3195180.61110659) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (10., 70.)>
Current Position is: 
 <ICRS Coordinate: (ra, dec) in deg
    (355.07323789, -10.61382164)>
Target Position is: 
 <AltAz Coordinate (obstime=58912.721776106686, location=(1819093.56876225, -5208411.6827961, -3195180.61110659) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (13.5, 73.5)>
Target Position is: 
 <ICRS Coordinate: (ra, dec) in deg
    (355.07323789, -10.61382164)>
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 52 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 12 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 13 messages
RemoteTelemetry(

In [10]:
# Why does doing no slew at all take ~32.7 sec?
start_time = time.time()
await attcs.slew_icrs(ra=str(ra_dec_target.ra), dec=str(ra_dec_target.dec), pa_ang=0.0, slew_timeout=240.)
end_time = time.time()
print('Time to slew is {} [s]'.format(end_time-start_time))

RemoteEvent(ATMCS, 0, target) falling behind; read 59 messages
Parallactic angle: -171.83468565676506 | Sky Angle: -121.83117517924009
Sending command
Stop tracking.
Mount tracking state is 2
Tracking state: 3.
Tracking state: 1.
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = -000.001 | delta Az= -000.185
[Telescope] delta Alt = -000.001 | delta Az= +000.012
[Telescope] delta Alt = +000.000 | delta Az= -000.000
Axes in position.
None
Time to slew is 35.18989062309265 [s]


In [36]:
# get RA/DEC of offseted position
radec = ICRS(ra, dec)
time_data = await attcs.atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

NameError: name 'ra' is not defined